In [ ]:
import mpld3
import numpy as np
from IPython.display import display
from matplotlib import pyplot as plt
from pydrake.all import PiecewisePolynomial, ZmpPlanner

from underactuated import running_as_notebook

if running_as_notebook:
    mpld3.enable_notebook()

This example shows the basics of working with the ZmpPlanner.  Try changing the footstep timing; see what happens to the resulting CoM trajectory.

In [ ]:
def GenerateDesiredZmpTraj(
    footsteps: np.typing.NDArray,
    double_support_duration: float,
    single_support_duration: float,
) -> PiecewisePolynomial:
    """Computes a desired ZMP trajectory from a sequence of footsteps.

    Args:
        footsteps: An array of shape (n, 2) where n is the number of footsteps.
        double_support_duration: The duration of the double support phase.
        single_support_duration: The duration of the single support phase.
    """

    time_steps = []
    zmp_d = []

    time = 0

    time_steps.append(time)
    zmp_d.append(footsteps[0])
    time += single_support_duration
    time_steps.append(time)
    zmp_d.append(footsteps[0])

    for i in range(1, len(footsteps)):
        time += double_support_duration
        time_steps.append(time)
        zmp_d.append(footsteps[i])

        time += single_support_duration
        time_steps.append(time)
        zmp_d.append(footsteps[i])

    zmp_d = np.array(zmp_d).T

    # zmp_traj = PiecewisePolynomial.ZeroOrderHold(time_steps, zmp_d)
    zmp_traj = PiecewisePolynomial.FirstOrderHold(time_steps, zmp_d)
    # zmp_traj = PiecewisePolynomial.CubicShapePreserving(time_steps, zmp_d)

    return zmp_traj


def example():
    com0 = np.array([0, 0])
    comdot0 = np.array([0, 0])
    x0 = np.concatenate((com0, comdot0))
    height = 1

    # an average human walking stride length is about 0.7m, and width is about 0.1m
    footsteps = np.array(
        [[0, -0.05], [0.5, 0.05], [1, -0.05], [1.5, 0.05], [2, -0.05], [2.5, 0.05]]
    )

    zmp_traj = GenerateDesiredZmpTraj(
        footsteps, double_support_duration=0.5, single_support_duration=1
    )

    zmp_planner = ZmpPlanner()
    zmp_planner.Plan(zmp_traj, x0, height)

    N = 100
    ts = np.linspace(0, zmp_traj.end_time(), 100)
    desired_zmp = np.zeros((N, 2))
    nominal_com = np.zeros((N, 2))

    plt.figure(figsize=(10, 6))
    # Draw the footsteps
    for i in range(len(footsteps)):
        # an average foot is 0.3m long and 0.1m wide
        plt.fill(
            footsteps[i, 0] + [-0.15, 0.15, 0.15, -0.15],
            footsteps[i, 1] + [-0.05, -0.05, 0.05, 0.05],
            "lightgray",
        )

    for i, t in enumerate(ts):
        desired_zmp[i] = zmp_planner.get_desired_zmp(t)
        nominal_com[i] = zmp_planner.get_nominal_com(t)

    plt.plot(desired_zmp[:, 0], desired_zmp[:, 1], "r", label="Desired ZMP")
    plt.plot(nominal_com[:, 0], nominal_com[:, 1], "b", label="Nominal CoM")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")
    plt.ylim(-0.3, 0.3)
    plt.gca().set_aspect("equal")
    plt.legend()

    display(mpld3.display())


example()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a45cd6cf-c8cd-4212-9b4f-261528e6e6af' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>